# Miscellaneous from Youtube, Google, ... etc

## Generator : Performance

generator는 변수를 메모리에 저장하지 않고 바로 리턴하기 때문에  
메모리, 시간 면에서 return으로 데이터를 반환하는 것보다 효율적이다.  

파이썬 3.8에서 time.clock()은 사라질 것이니 process_time, perf_counter 등을 사용하라고 한다.

In [1]:
import random, time
import memory_profiler as mem_profile

In [2]:
names = ['John', 'Corey', 'Adam', 'Steve', 'Rick', 'Thomas']
majors = ['Math', 'Engineering', 'CompSci', 'Arts', 'Business']

In [3]:
print("Memory (Before): {}Mb".format(mem_profile.memory_usage()))

Memory (Before): [46.08203125]Mb


In [4]:
def people_list(num_people):
    result = []
    for i in range(num_people):
        person = {
            'id': i,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        result.append(person)
    return result

In [5]:
def people_generator(num_people):
    for i in range(num_people):
        person = {
            'id': i,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        yield person

In [6]:
t1 = time.process_time()
people = people_list(1000000)
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [339.546875]Mb
Took 1.399382 Seconds


In [7]:
t1 = time.process_time()
people = people_generator(1000000)
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [50.08203125]Mb
Took 0.09962199999999988 Seconds


In [8]:
t1 = time.process_time()
people = list(people_generator(1000000))
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [341.9765625]Mb
Took 1.4080149999999998 Seconds


<hr>

## Variable Scope

**LEGB**  
**L**ocal (variables defined in functions)  
**E**nclosing (variables defined in the local scope of enclosing functions)  
**G**lobal (variables defined at the top level of a module or global keyword)  
**B**uilt-in (variables pre assigned in Python)

위의 순서 = 파이썬이 변수의 영역을 체크하는 순서

#### global vs. local

In [9]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    print(y)
    
test()

local y


test() 함수에서 x를 출력하는 것 : global 변수 출력  
First, find local scope  

In [10]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    print(x) # global x
    
test()

global x


In [11]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    
test()
# y 변수는 test() 함수 바깥에 존재하지 않음 (Error)
print(y)

NameError: name 'y' is not defined

test() 함수에서 x를 over-write 하지 않는다.

In [13]:
x = 'global x'

def test():
    x = 'local x'
    print(x)
    
test()
print(x)

local x
global x


global 키워드 : global scope의 변수를 사용함

In [14]:
x = 'global x'

def test():
    global x
    x = 'local x'
    print(x)
    
test()
print(x)

local x
local x


global 영역에 존재하지 않는 변수는 생성해서 에러 발생 안 함

In [15]:
# x = 'global x'

def test():
    global x
    x = 'local x'
    print(x)
    
test()
print(x)

local x
local x


global 키워드를 자주 사용하지 않는다.  
만약 자주 사용한다면, 함수 외부에서 오버라이딩 되는 것을 매우 고려해야함

In [16]:
# z 변수도 local scope 에 존재함
# 차이점은 함수에 값을 전달하는 함수 파라미터라는 것
def test(z):
    x = 'local x'
    print(z)
    
test('local z')

local z


In [17]:
def test(z):
    x = 'local x'
    print(z)
    
test('local z')
# z is only local to test function
print(z)

local z


NameError: name 'z' is not defined

#### built-in scope  
python에 미리 할당된 이름들

In [18]:
m = min([5, 1, 4, 3, 2]) # built-in function
print(m)

1


In [19]:
import builtins

# dir() : 객체의 속성들의 리스트
print(dir(builtins))

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

내장 함수, 변수는 오버라이딩이 가능함  
built-in scope보다 global scope에서 먼저 변수, 함수를 찾기 때문에 가능함  
따라서 주의해라

In [20]:
def min():
    pass

In [21]:
min([2, 1, 3])

TypeError: min() takes 0 positional arguments but 1 was given

#### enclosing

nested function

In [22]:
def outer():
    # local x to outer func
    x = 'outer x'
    
    def inner():
        # local x to inner func
        x = 'inner x'
        print(x)  # inner x
    
    inner()
    print(x)  # outer x
    
outer()

inner x
outer x


enclosing scope  
= enclosing function의 local scope

In [23]:
def outer():
    # local x to outer func
    x = 'outer x'
    
    def inner():
        # check local x -> doesn't exist
        # check enclosing x -> exists
        # inner's enclosing func. is outer func.
        print(x)  # outer x
    
    inner()
    print(x)  # outer x
    
outer()

outer x
outer x


In [25]:
def outer():
    
    def inner():
        a = 'inner a'
        print(a)  # inner a
    
    inner()
    print(a)  # Error
    
outer()

inner a


NameError: name 'a' is not defined

enclosing scope의 변수를 변경하려면?  
nonlocal 키워드  

global 변수보다 자주 사용된다  
클로저, 데코레이터의 상태를 변경하기 위해

In [26]:
# inner func.에서 enclosing x 를 변경하려면?
def outer():
    x = 'outer x'
    
    def inner():
        nonlocal x
        x = 'inner x'  # overwrite
        print(x)  # inner x
    
    inner()
    print(x)  # inner x
    
outer()

inner x
inner x


In [27]:
x = 'global x'

def outer():
    x = 'outer x'
    
    def inner():
        x = 'inner x'
        print(x)
        
    inner()
    print(x)

outer()
print(x)

inner x
outer x
global x


<hr>

## First-Class Functions

Wikipedia에 의한 First-Class Functions :  
first-class citizen으로 함수를 다루는 것(treat)

First-Class Citizen (Programming) :  
an entity which supports all the operations(being passed as an argument, returned from a function, and assigned to a variable) generally available to other entities  
(함수의 매개변수로 전달 가능하고, 함수의 반환값으로 반환될 수 있고, 어떤 변수에 할당가능한 개체)

함수를 객체나 변수처럼 다룰 수 있다

In [28]:
def square(x):
    return x * x

f = square(5)

print(square)
print(f)

<function square at 0x7f93ae2e0170>
25


함수명에 () 소괄호를 붙이는 것 = 함수를 실행한다는 의미

In [30]:
f = square # 실행 X, 함수자체를 의미

In [31]:
print(square)
print(f)

<function square at 0x7f93ae2e0170>
<function square at 0x7f93ae2e0170>


In [32]:
print(f(5))

25


대표적으로 map 함수  
함수를 인자로 전달받는다.

In [33]:
def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

squares = my_map(square, [1, 2, 3, 4, 5])

print(squares)

[1, 4, 9, 16, 25]


In [34]:
def cube(x):
    return x * x * x

cubes = my_map(cube, [1, 2, 3, 4, 5])

print(cubes)

[1, 8, 27, 64, 125]


함수를 리턴하는 간단한 예제

In [35]:
def logger(msg):
    
    def log_message():
        print('Log:', msg)
        
    return log_message

log_hi = logger('Hi!')  # log_message 함수
log_hi()

Log: Hi!


practical에 가까운 example

In [37]:
def html_tag(tag):
    
    def wrap_text(msg):
        print("<{0}>{1}</{0}>".format(tag, msg))
        
    return wrap_text

print_h1 = html_tag('h1')
print_h1('Test Headline')
print_h1('Another Headline')

print_p = html_tag('p')
print_p('Test Paragraph')

<h1>Test Headline</h1>
<h1>Another Headline</h1>
<p>Test Paragraph</p>


*Ref. https://youtu.be/kr0mpwqttM0*

<hr>

## Closures

Wikipedia 에 의한 Closure :  
A record storing a function together with an environment  

In [38]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        # message : free variable (inner_func에 정의되어 있지 않지만 접근 가능한 변수)
        print(message)
        
    return inner_func()

outer_func()

Hi


In [39]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        print(message)
        
    return inner_func

my_func = outer_func()

print(my_func)
print(my_func.__name__)

<function outer_func.<locals>.inner_func at 0x7f93ae76b9e0>
inner_func


In [40]:
"""
easy/simple meaning of closure
: can access to variables in the local scope in which it was created
"""
my_func()
my_func()
my_func()

Hi
Hi
Hi


closure closes over the free variables from their environment  
message is a free variable

In [41]:
def outer_func(msg):
    message = msg
    
    def inner_func():
        print(message)
        
    return inner_func

hi_func = outer_func('Hi')
hello_func = outer_func('Hello')

In [42]:
hi_func()

Hi


In [43]:
hello_func()

Hello


In [44]:
import logging

logging.basicConfig(filename='example.log', level=logging.INFO)

def logger(func):
    def log_func(*args):
        logging.info('Running "{}" with arguments {}'.format(func.__name__, args))
        print(func(*args))
    return log_func

def add(x, y):
    return x+y

def sub(x, y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3, 3)
add_logger(4, 5)

sub_logger(10, 5)
sub_logger(20, 10)

6
9
5
10


*Ref. https://youtu.be/swU3c34d2NQ*

<hr>

## Decorators